In [74]:
# Libraries

import warnings
warnings.filterwarnings('ignore')

# b) Data extraction
import os, base64
import sys
import xml.etree.ElementTree as ET
import rdflib
import re
from lxml import etree

# c) General
import copy
# import difflib
import numpy as np
from numpyencoder import NumpyEncoder
import pandas as pd
import math
import operator as op
# import ast
import re
# import string
# import csv
import xlsxwriter
import json
import ipywidgets as widgets
import ipyvuetify
from ipywidgets import interact, interact_manual, interactive, IntSlider, FloatSlider
from IPython.display import display, clear_output, Image, FileLink, HTML
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from scipy.optimize import curve_fit
# from github import Github
import requests
from scipy.optimize import least_squares

# d) Plot
import matplotlib.pyplot as plt
from matplotlib import markers
import matplotlib.font_manager as font_manager
import matplotlib.colors
from pylab import rcParams
import matplotlib.image as mpimg


In [75]:
def singleKineticBuilder(bgTemplates):

    speciesNoDuplicate = []
    for reaction in bgTemplates[singleSelection.value]['products']:
        for stoi,product in bgTemplates[singleSelection.value]['products'][reaction]:
            if product not in speciesNoDuplicate:
                speciesNoDuplicate.append(product)
    for reaction in bgTemplates[singleSelection.value]['reactants']:
        for stoi,reactant in bgTemplates[singleSelection.value]['reactants'][reaction]:
            if reactant not in speciesNoDuplicate:
                speciesNoDuplicate.append(reactant)
        
  
    N=np.zeros((len(speciesNoDuplicate),len(bgTemplates[singleSelection.value]['products'])))
    
    for s in range(len(speciesNoDuplicate)):
        for i,reaction in enumerate(bgTemplates[singleSelection.value]['products']):
            for stoi,product in bgTemplates[singleSelection.value]['products'][reaction]:
                if speciesNoDuplicate[s]==product:
                    N[s][i] = float(stoi)
        for i,reaction in enumerate(bgTemplates[singleSelection.value]['reactants']):
            for stoi,reactant in bgTemplates[singleSelection.value]['reactants'][reaction]:
                if speciesNoDuplicate[s]==reactant:
                    N[s][i] = -float(stoi)
    
    
    return [N,speciesNoDuplicate]


In [76]:
def textFileGen(speciesNoDuplicate,bgTemplates,kfkr_estimated,floatICget,N):

    # Create a text file (TEXT)

    unitsImport = ['def model composed_model as',
                   '\tdef import using "units_BG.cellml" for',
                    '\t\tunit concentrationUnit using unit concentrationUnit;',
                   '\t\tunit speciesConstantUnit using unit speciesConstantUnit;',
                   '\t\tunit fluxUnit using unit fluxUnit;',
                    '\tenddef;',
                  '\tdef comp main as']
    

    with open('GFG.txt', 'w') as fp:
        pass    
    with open('GFG.txt', 'a') as file:
        for line in unitsImport:
            file.write(line+'\n')


    with open('GFG.txt', 'a') as file:
        file.write('\t\tvar t: second {init: 0};'+'\n')


    for i,species in enumerate(speciesNoDuplicate):
        with open('GFG.txt', 'a') as file:
            file.write('\t\tvar ' +species+': '+'concentrationUnit '+'{'+'init: '+str([x.value for x in floatICget if x.description==species][0])+'}'+';'+'\n')


    for reaction in bgTemplates[singleSelection.value]['reactants']:
        with open('GFG.txt', 'a') as file:
            file.write('\t\tvar '+reaction+': '+'fluxUnit '+';'+'\n')


    for k in kfkr_estimated[singleSelection.value]:
        with open('GFG.txt', 'a') as file:
            file.write('\t\tvar '+ k+': '+'fluxUnit '+'{'+'init: '+str(kfkr_estimated[singleSelection.value][k])+'}'+';'+'\n')


    # Generating the ODEs       
    with open('GFG.txt', 'a') as file:
        file.write('\n')

    for i,species in enumerate(speciesNoDuplicate):
        with open('GFG.txt', 'a') as file:
            file.write('\t\tode('+species+',t)=')
        for j in range(np.shape(N)[1]):  
            if N[i][j] != 0:

                if N[i][j] > 0:
                    with open('GFG.txt', 'a') as file:
                        file.write('+'+str(N[i][j])+'{dimensionless}*'+list(bgTemplates[singleSelection.value]['products'].keys())[j])

                if N[i][j] < 0:
                    with open('GFG.txt', 'a') as file:
                        file.write(str(N[i][j])+'{dimensionless}*'+list(bgTemplates[singleSelection.value]['products'].keys())[j])

        with open('GFG.txt', 'a') as file:
            file.write(';\n')


    # Generating the reaction rates
    with open('GFG.txt', 'a') as file:
        file.write('\n')

    for r,flux in enumerate(bgTemplates[singleSelection.value]['products'].keys()):
        with open('GFG.txt', 'a') as file:
            file.write('\t\t '+ flux+'= ') 
                    
        
#         if singleSelection.value=='SGLT1' and 'sodium(1+)' in meaning[flux+'_'+singleSelection.value]:
#             z_Na=1; F=96485; T=310; R=8.314; V_na=-0.046;
#             membraneContrib = math.exp((z_Na*F*V_na)/(R*T))
#             with open('GFG.txt', 'a') as file:
#                 file.write(str(membraneContrib)+'{dimensionless}'+'*')

        
        with open('GFG.txt', 'a') as file:
            file.write('k_'+str(r+1)+'f'+'*(')
        length = 0    
        for stoi,reactant in bgTemplates[singleSelection.value]['reactants'][flux]:
            length+=1
            with open('GFG.txt', 'a') as file:
                file.write('pow('+reactant+','+str(stoi)+'{dimensionless}'+')') 

            if length < len(bgTemplates[singleSelection.value]['reactants'][flux]):
                with open('GFG.txt', 'a') as file:
                    file.write(' * ')



        with open('GFG.txt', 'a') as file:
            file.write(') - ')
            
            
        with open('GFG.txt', 'a') as file:
            file.write('k_'+str(r+1)+'r'+'*(') 
        length = 0
        for stoi,product in bgTemplates[singleSelection.value]['products'][flux]:
            length+=1
            with open('GFG.txt', 'a') as file:
                file.write('pow('+product+','+str(stoi)+'{dimensionless}'+')') 

            if length < len(bgTemplates[singleSelection.value]['products'][flux]):
                with open('GFG.txt', 'a') as file:
                    file.write(' * ')

        with open('GFG.txt', 'a') as file:
            file.write(');\n')


        





    with open('GFG.txt', 'a') as file:
        file.write('\tenddef;'+'\n'+'enddef;')
        
    return 

In [77]:
# Generating a json file for BG templates
F=96485
type2_z_B=1

types = ['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9']
reactions = {'type 1':['Re1', 'Re2', 'Re3', 'Re4'], 
            'type 2':['Re1', 'Re2', 'Re3', 'Re4', 'Re5', 'Re6'], 
            'type 3':['Re1', 'Re2', 'Re3', 'Re4', 'Re5', 'Re6'], 
            'type 4':['Re1', 'Re2', 'Re3', 'Re4', 'Re5', 'Re6', 'Re7', 'Re8']}

X = dict((el,dict((el1,{}) for el1 in ['reactants','products'])) for el in types)


X['type 1']['reactants']={'Re1': [(1,'EAi')], 'Re2': [(1,'Eo'),(1,'Ao')], 'Re3': [(1,'Ei')], 'Re4': [(1,'EAo')]}
X['type 1']['products']={'Re1': [(1,'Ai'),(1,'Ei')], 'Re2': [(1,'EAo')], 'Re3': [(1,'Eo')], 'Re4': [(1,'EAi')]}

X['type 2']['reactants']={'Re1': [(1,'Eo'),(1,'Ao')], 'Re2': [(1,'EAi')], 'Re3': [(1,'EAo'),(1,'Bo'),(type2_z_B*F,'Vm')], 
                          'Re4': [(1,'EBAi')], 'Re5': [(1,'EBAo')], 'Re6': [(1,'Ei')]}
X['type 2']['products']={'Re1': [(1,'EAo')], 'Re2': [(1,'Ai'),(1,'Ei')], 'Re3': [(1,'EBAo')], 
                         'Re4': [(1,'EAi'),(1,'Bi')], 'Re5': [(1,'EBAi')], 'Re6': [(1,'Eo')]}

X['type 3']['reactants']={'Re1': [(1,'Eo'),(1,'Ao')], 'Re2': [(1,'EAi')], 'Re3': [(1,'EBo')], 
                          'Re4': [(1,'Bi'),(1,'Ei')], 'Re5': [(1,'EBi')], 'Re6': [(1,'EAo')]}
X['type 3']['products']={'Re1': [(1,'EAo')], 'Re2': [(1,'Ai'),(1,'Ei')], 'Re3': [(1,'Bo'),(1,'Eo')], 
                         'Re4': [(1,'EBi')], 'Re5': [(1,'EBo')], 'Re6': [(1,'EAi')]}

X['type 4']['reactants']={'Re1': [(1,'EAo')], 'Re2': [(1,'Ai'),(1,'Ei')], 'Re3': [(1,'Eo'),(1,'Bo')], 'Re4': [(1,'EBi')], 
                          'Re5': [(1,'EBo'),(2,'Co')], 'Re6': [(1,'EBCi')], 'Re7': [(1,'EAi')], 'Re8': [(1,'EBCo')]}
X['type 4']['products']={'Re1': [(1,'Ao'),(1,'Eo')], 'Re2': [(1,'EAi')], 'Re3': [(1,'EBo')], 'Re4': [(1,'Ei'),(1,'Bi')], 
                         'Re5': [(1,'EBCo')], 'Re6': [(1,'EBi'),(2,'Ci')], 'Re7': [(1,'EAo')], 'Re8': [(1,'EBCi')]}
    

json_object = json.dumps(X, indent=4, sort_keys=True,
      separators=(', ', ': '), ensure_ascii=False,
      cls=NumpyEncoder)
with open("./jsonTemplates/bgTemplates.json", "w") as outfile:
    outfile.write(json_object)

## Section 1: Model selection


In [78]:
# Model Selection and BG structure demonstration
singleSelection = widgets.Dropdown(
            options=['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9'],
            description='Select BG type:',
            style={'description_width': 'initial'},
            disabled=False
        )
button_load = widgets.Button(
                description='Load template',
                tooltip='Load',
                style={'description_width': 'initial'},
                button_style='primary'
            )
output1 = widgets.Output(layout={'border': '1px solid black'})

def button_load_clicked(event):
    with output1:
        clear_output()

        # Define the path of the folder containing the images you want to display
        folder_path = './figures'

        for file in os.listdir(folder_path):
            if file.strip('.jpg') == singleSelection.value or \
            file.strip('.JPG') == singleSelection.value or \
            file.strip('.JPEG') == singleSelection.value or \
            file.strip('.jpeg') == singleSelection.value: 
                file_path = os.path.join(folder_path, file)
                img = Image.open(file_path)
                new_image = img.resize((800, 500))
                text_0 = widgets.HTML(value="<h5><b>Bond graph structure of {}:<b><h5>".format(file.strip('.jpg')))
                display(text_0,new_image)

        
button_load.on_click(button_load_clicked)
vbox_result = widgets.VBox([button_load, output1])   

vbox_text = widgets.VBox([singleSelection, vbox_result])
page1 = widgets.HBox([vbox_text])

display(page1)

## Section 2: Insert the Initial Conditions


In [79]:
# Getting the initial conditions
types = ['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9']

valueRequired1 = dict((el,[]) for el in types)
valueRequired1['type 1']=['Ai','Ao']
valueRequired1['type 2']=['Ai','Ao','Bi','Bo','z_B','Vm']
valueRequired1['type 3']=['Ai','Ao','Bi','Bo']
valueRequired1['type 4']=['Ai','Ao','Bi','Bo','Ci','Co']

Enzymes = dict((el,[]) for el in types)
Enzymes['type 1']=['Ei','Eo','EAi','EAo']
Enzymes['type 2']=['Ei','Eo','EAi','EAo','EBAo','EBAi']
Enzymes['type 3']=['Ei','Eo','EAi','EAo','EBo','EBi']
Enzymes['type 4']=['Ei','Eo','EAi','EAo','EBo','EBi','EBCo','EBCi']


E_tot = widgets.FloatLogSlider(
    value=1e-6,
    base=10,
    min=-6, # max exponent of base
    max=0, # min exponent of base
    step=0.1, # exponent step
    description='Total Enzyme Concentration',
    layout={'width': 'auto'},
    style={'description_width': 'initial'},
)
E_tot.style.handle_color = 'lightblue'
display(E_tot)
button_saveEtot = widgets.Button(
                description='Save',
                tooltip='Save',
                style={'description_width': 'initial'},
                button_style='success'
            )
output7 = widgets.Output(layout={'border': '1px solid black'})



button_saveIC = widgets.Button(
                description='Confirm',
                tooltip='Save',
                style={'description_width': 'initial'}
            )
button_saveIC.style.button_color = 'lightgreen'
def button_saveIC_clicked(button_saveIC):
    with output7:
        clear_output()
        print('The total enzyme concentration has been saved.')
        print('Insert the initial concentrations.')

        global E_tot, floatICget
        floatICget=[]
        for i,el in enumerate(valueRequired1[singleSelection.value]):
            floatICget.append(widgets.FloatText(
                    value=0.0,
                    description=el,
                    disabled=False
                ))
        for i,el in enumerate(Enzymes[singleSelection.value]):
            floatICget.append(widgets.FloatText(
                    value=E_tot.value/len(Enzymes[singleSelection.value]),
                    description=el,
                    disabled=False
                ))
            
        output8 = widgets.Output(layout={})
        
        display(*floatICget)
        def button_saveEtot_clicked(event):
            with output8:
                clear_output()
                text = widgets.HTML(value="<h5><b>The initial conditions have been saved.<b><h5>")
                display(text)   
                
        button_saveEtot.on_click(button_saveEtot_clicked)

        display(button_saveEtot)
        display(output8)

button_saveIC.on_click(button_saveIC_clicked)
    
    
vbox_result = widgets.VBox([button_saveIC, output7])   

page5 = widgets.HBox([vbox_result])

display(page5)
   




FloatLogSlider(value=1e-06, description='Total Enzyme Concentration', layout=Layout(width='auto'), max=0.0, mi…

## Section 3: Insert the Steady-State Values



In [80]:
# Getting the SS values
types = ['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9']
valueRequired = dict((el,[]) for el in types)

valueRequired['type 1']=['V_SS','Ai','Ao']
valueRequired['type 2']=['V_SS','Ai','Ao','Bi','Bo']
valueRequired['type 3']=['V_SS','Ai','Ao','Bi','Bo']
valueRequired['type 4']=['V_SS','Ai','Ao','Bi','Bo','Ci','Co']

floatSSget=[]
for i,el in enumerate(valueRequired[singleSelection.value]):
    floatSSget.append(widgets.FloatText(
            value=0.0,
            description=el,
            disabled=False
        ))
button_saveSS = widgets.Button(
                description='Save',
                tooltip='Save',
                style={'description_width': 'initial'},
                button_style='success'
            )
output2 = widgets.Output(layout={'border': '1px solid black'})

def button_saveSS_clicked(event):
    with output2:
        clear_output()
        text = widgets.HTML(value="<h5><b>The inserted steady-state values have been saved.<b><h5>")
        display(text)      

button_saveSS.on_click(button_saveSS_clicked)
display(*floatSSget)

vbox_result = widgets.VBox([button_saveSS, output2])   
page6 = widgets.HBox([vbox_result])
display(page6)




FloatText(value=0.0, description='V_SS')

FloatText(value=0.0, description='Ai')

FloatText(value=0.0, description='Ao')

## Section 4: The Steady-State Parameters


In [81]:
button_pVals = widgets.Button(
                description='Estimate the steady-state parameters',
                tooltip='Description',
                layout={'width': '300px'},
                style={'description_width': 'initial'},
                button_style='primary'
            )
output3 = widgets.Output(layout={'border': '1px solid black'})


def button_pVals_clicked(event):
    with output3:
        clear_output()
        # This section works for 'type 1'
        if singleSelection.value == 'type 1':
            

            solve = tuple()
            solve = solve+([el.value for el in floatSSget if el.description=='Ai'][0],)
            solve = solve+([el.value for el in floatSSget if el.description=='Ao'][0],)


            bounds=[[],[]]
            for i in range(6):
                bounds[0].append(0)
                bounds[1].append(np.inf)



            def func(X, p1,p2,p3,p4,p5,p6):
                Ai,Ao = X

                V_SS = E_tot.value* (p5* Ao - p6* Ai)/ (p1 + p2*Ao + p3*Ai*Ao + p4*Ai)

                return V_SS


            popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'][0], maxfev=3000000, bounds=bounds)
            if all(x==1 for x in popt): 
                print('Convergence error: Increase the E_tot value.')


            pVals={}
            for i,el in enumerate(popt):
                pVals['p'+str(i+1)]=el
            print(pVals)
                
            

         # Create the export button widget
        button_export = widgets.Button(description='Export the values')
        button_export.style.button_color = 'lightgreen'
        
        output4 = widgets.Output()
        # Define a function to handle the button click event
        def export_values(button_export):
            with output4:
                clear_output()
                # Save the values in .json file
                json_object = json.dumps(pVals, indent=4, sort_keys=True,
                      separators=(', ', ': '), ensure_ascii=False,
                      cls=NumpyEncoder)
                with open("./temporary files/pVals.json", "w") as outfile:
                    outfile.write(json_object)
                text_2 = widgets.HTML(value="<h5><b>Estimated steady-state parameters saved to pVals.json<b><h5>")
                display(text_2)

        # Attach the save_number function to the button click event
        button_export.on_click(export_values)
        # Display the widgets
        vbox_result = widgets.VBox([button_export, output4]) 
        page4 = widgets.HBox([vbox_result])
        display(page4)

      
        
button_pVals.on_click(button_pVals_clicked)
vbox_result = widgets.VBox([button_pVals, output3]) 

page3 = widgets.HBox([vbox_result])

display(page3)



## Section 5: The Kinetic Parameters


In [82]:
button_kfkr = widgets.Button(
                description='Estimate the kinetic parameters',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout={'width': 'auto'},
                button_style='primary'
            )
output5 = widgets.Output(layout={'border': '1px solid black'})

def button_kfkr_clicked(event):
    with output5:
        clear_output()
        f = open("./temporary files/pVals.json")
        pVals = json.load(f)
        
        if singleSelection.value == 'type 1':
            # Define the function that returns the difference between 
            # the equations and the known values
            def equations(variables, *data):
                k_1f, k_2f, k_3f, k_4f, k_1r, k_2r, k_3r, k_4r = variables
                p1, p2, p3, p4, p5, p6, p7, p8 = data

                eq1 = k_1f * k_3r * k_4f + k_1f * k_2r * k_3r + k_2r * k_3r * k_4r - p1*k_3f*k_4f
                eq2 = k_1f * k_2f * k_4f + k_2f * k_3f * k_4f + k_1f * k_2f * k_3f - p2*k_3r*k_4r
                eq3 = k_1r * k_2f * k_4f + k_1r * k_2f * k_4r - p3*k_1f
                eq4 = k_1r * k_2r * k_4r + k_1r * k_3r * k_4f + k_1r * k_2r * k_3r + k_1r * k_3r * k_4r - p4*k_1f*k_2f
                eq5 = k_1f * k_2f * k_3f * k_4f - p5
                eq6 = k_1r * k_2r * k_3r * k_4r - p6
                eq7 = k_1f + k_1r - p7
                eq8 = k_2f + k_2r - p8

                return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8]

            # Set the known values
            p1, p2, p3, p4, p5, p6, p7, p8 = [x for x in pVals.values()]+ [1, 1]

            # Call fsolve to find the values of the unknown variables
            x0 = (p1/p5, p2/p5, p3/p5, p4/p5, p6/p5, p7, p8, p7) # initial guess for variables
            bounds = tuple()
            for i in range(len(x0)):
                bounds = bounds+((0, np.inf),) # bounds for variables

            sol = least_squares(equations, x0, args=(p1, p2, p3, p4, p5, p6, p7, p8), bounds=(0, np.inf))

            # Print the solution
            print("k_1f =", sol.x[0])
            print("k_2f =", sol.x[1])
            print("k_3f =", sol.x[2])
            print("k_4f =", sol.x[3])
            print("k_1r =", sol.x[4])
            print("k_2r =", sol.x[5])
            print("k_3r =", sol.x[6])
            print("k_4r =", sol.x[7])
            
            kfkr_estimated={}
            keys=['k_1f', 'k_2f', 'k_3f', 'k_4f', 'k_1r', 'k_2r', 'k_3r', 'k_4r']
            kfkr_estimated['type 1'] = dict((el,sol.x[i]) for el,i in zip(keys,range(len(sol.x))))
            json_object = json.dumps(kfkr_estimated, indent=4, sort_keys=True,
                      separators=(', ', ': '), ensure_ascii=False,
                      cls=NumpyEncoder)
            with open("./temporary files/kfkr_estimated.json", "w") as outfile:
                outfile.write(json_object)

button_kfkr.on_click(button_kfkr_clicked)
vbox_result = widgets.VBox([button_kfkr, output5]) 

page5 = widgets.HBox([vbox_result])

display(page5)

## Section 6: CellML Code


In [ ]:
button_cellml = widgets.Button(
                description='Generate code',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='danger'
            )
output6 = widgets.Output(layout={'border': '1px solid black'})

def on_button_clicked6(event):
    with output6:
        clear_output()
        f = open("./temporary files/kfkr_estimated.json")
        kfkr_estimated = json.load(f)
        f = open("./jsonTemplates/bgTemplates.json")
        bgTemplates = json.load(f)
        
        [N,speciesNoDuplicate] = singleKineticBuilder(bgTemplates)
        
        textFileGen(speciesNoDuplicate,bgTemplates,kfkr_estimated,floatICget,N)
        text_0 = widgets.HTML(value="<h3>The CellML code for the bond graph form of {} is:</h3>".format(singleSelection.value))
        vbox_text = widgets.VBox([text_0])
        display(vbox_text)
        with open('GFG.txt') as f:
            contents = f.read()
        print(contents)

button_cellml.on_click(on_button_clicked6)
vbox_result = widgets.VBox([button_cellml, output6]) 
display(vbox_result)


# Section X: Composite Annotation

In [184]:
import pandas as pd
import ipywidgets as widgets 
from ipywidgets import Layout


def annotate():
    # Create a text input widget for searching
    search_widget_left = widgets.Text(
        placeholder='Enter search term...',
        description='Keyword:'
    )
    search_widget_middle = widgets.Text(
        placeholder='Enter search term...',
        description='Keyword:'
    )
    search_widget_right = widgets.Text(
        placeholder='Enter search term...',
        description='Keyword:'
    )
    
    # Create radio buttons for selecting from retrieved values
    radio_buttons_left = widgets.RadioButtons(
        options=[],
        description='Select from retrieved values:',
        disabled=True,
        style={'description_width': 'initial'}
    )
    radio_buttons_middle = widgets.RadioButtons(
        options=[],
        description='Select from retrieved values:',
        disabled=True,
        style={'description_width': 'initial'}
    )
    radio_buttons_right = widgets.RadioButtons(
        options=[],
        description='Select from retrieved values:',
        disabled=True 
    )


    # Create a dropdown widget for selecting the search column
    column_names_left = ['Preferred Label','Class ID']
    column_names_widget_left = widgets.Dropdown(
        options=['Preferred Label','Class ID'],
        value='Preferred Label',
        description='Search column:',
        style={'description_width': 'initial'}
    )
    column_names_middle = ['Preferred Label','Class ID']
    column_names_widget_middle = widgets.Dropdown(
        options=['Preferred Label','Class ID'],
        value='Preferred Label',
        description='Search column:',
        style={'description_width': 'initial'}
    )
    column_names_right = ['Preferred Label','Class ID']
    column_names_widget_right = widgets.Dropdown(
        options=['Preferred Label','Class ID'],
        value='Preferred Label',
        description='Search column:',
        style={'description_width': 'initial'}
    )

    # Create an output widget for displaying search results
    output_widget_left = widgets.Output(layout={'height': '150px', 'overflow': 'scroll'})
    output_widget_middle= widgets.Output(layout={'height': '150px', 'overflow': 'scroll'})
    output_widget_right = widgets.Output(layout={'height': '150px', 'overflow': 'scroll'})

    # Define a function to handle search events
    def handle_search_left(sender1):
        with output_widget_left:
            output_widget_left.clear_output()
            df_left = pd.read_csv('OPB.csv')
            search_term_left = search_widget_left.value.lower()
            column_name_left = column_names_widget_left.value
            filtered_df_left = df_left[df_left[column_name_left].str.lower().str.contains(search_term_left)]
            radio_buttons_left.options = [z+':  '+y for z,y in zip([x.replace('http://bhi.washington.edu/OPB#',"") for x in filtered_df_left['Class ID'].tolist()],filtered_df_left['Preferred Label'].tolist())]
            radio_buttons_left.disabled = False
            radio_buttons_left.layout = Layout(width='100%', height='100%')
            display(radio_buttons_left)

    def handle_search_middle(sender2):
        with output_widget_middle:
            output_widget_middle.clear_output()
            df_middle = pd.read_csv('CHEBI.csv')
            search_term_middle = search_widget_middle.value.lower()
            column_name_middle = column_names_widget_middle.value
            filtered_df_middle = df_middle[df_middle[column_name_middle].astype(str).str.lower().str.contains(search_term_middle.lower(), na=False)]
            radio_buttons_middle.options = [z+':  '+y for z,y in zip([x.replace('http://purl.obolibrary.org/obo/',"") for x in filtered_df_middle['Class ID'].tolist()],filtered_df_middle['Preferred Label'].tolist())]
            radio_buttons_middle.disabled = False
            radio_buttons_middle.layout = Layout(width='100%', height='100%')
            display(radio_buttons_middle)

    def handle_search_right(sender3):
        with output_widget_right:
            output_widget_right.clear_output()
            df_right = pd.read_csv('FMA.csv')
            search_term_right = search_widget_right.value.lower()
            column_name_right = column_names_widget_right.value
            filtered_df_right = df_right[df_right[column_name_right].astype(str).str.lower().str.contains(search_term_right.lower(), na=False)]
            radio_buttons_right.options = [z+':  '+y for z,y in zip([x.replace('http://purl.obolibrary.org/obo/',"") for x in filtered_df_right['Class ID'].tolist()],filtered_df_right['Preferred Label'].tolist())]
            radio_buttons_right.disabled = False
            radio_buttons_right.layout = Layout(width='100%', height='100%')
            display(radio_buttons_right)

    # Attach the search function to the text input widget's submit event
    search_widget_left.on_submit(handle_search_left)
    search_widget_middle.on_submit(handle_search_middle)
    search_widget_right.on_submit(handle_search_right)

    # Display the widgets
    text_left=widgets.HTML(value="<h4><b><font color='salmon'>Physical property:<b><h4>")
    display(text_left)
    display(search_widget_left, column_names_widget_left)
    display(output_widget_left)
    text_middle=widgets.HTML(value="<h4><b><font color='salmon'>Physical entity:<b><h4>")
    display(text_middle)
    display(search_widget_middle, column_names_widget_middle)
    display(output_widget_middle)
    text_right=widgets.HTML(value="<h4><b><font color='salmon'>Contained in:<b><h4>")
    display(text_right)
    display(search_widget_right, column_names_widget_right)
    display(output_widget_right)

In [185]:
for element in valueRequired1[singleSelection.value]:
    text=widgets.HTML(value="<h3><b><font color='teal'>{}<b><h3>".format(element))
    display(text)
    annotate()


HTML(value="<h3><b><font color='teal'>Ai<b><h3>")

HTML(value="<h4><b><font color='salmon'>Physical property:<b><h4>")

Text(value='', description='Keyword:', placeholder='Enter search term...')

Dropdown(description='Search column:', options=('Preferred Label', 'Class ID'), style=DescriptionStyle(descrip…

Output(layout=Layout(height='150px', overflow='scroll'))

...

Output(layout=Layout(height='150px', overflow='scroll'))

HTML(value="<h4><b><font color='salmon'>Contained in:<b><h4>")

Text(value='', description='Keyword:', placeholder='Enter search term...')

Dropdown(description='Search column:', options=('Preferred Label', 'Class ID'), style=DescriptionStyle(descrip…

Output(layout=Layout(height='150px', overflow='scroll'))

In [111]:
import ipywidgets as widgets
from IPython.display import display

# Define a simple function
def greet(name):
    print(f'Hello, {name}!')

# Create the widgets
name_widget = widgets.Text(description='Name:', placeholder='Enter your name')
button_widget = widgets.Button(description='Greet')
output_widget = widgets.Output()

# Define the button click event handler
def on_button_click(b):
    with output_widget:
        output_widget.clear_output()
        greet(name_widget.value)

# Attach the button click event handler
button_widget.on_click(on_button_click)

# Create the layout
left_box = widgets.VBox([name_widget, button_widget])
right_box = widgets.VBox([output_widget])
layout = widgets.HBox([left_box, right_box])

# Display the layout
display(layout)
